В этом ноутбуке разбивал данные на фолды

In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/sample_submission.csv
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/description.xlsx
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_target.csv
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/test_target.csv
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/test_data/test_data_0.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/test_data/test_data_1.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_0.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_9.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_5.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_4.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_2.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for

### 0. Imports and requirements

* Для моделирования нам понадобится реализация градиентного бустинга из `lightgbm`. Для обработки данных &ndash; стандартный DS стек из `pandas`, `numpy` и `sklearn`. Так как мы будем использовать достаточно объемные выборки данных (кредитные истории клиентов), то нужно будет эффективно читать данные и выделять из них признаки, чтобы иметь возможность строить решение даже на локальных машинах с ограничениями по оперативной памяти.  Поэтому нам понадобятся инструменты для работы с форматом данных `Parquet`. Библиотека `pandas` представляет необходимый инструментарий (рекомендуем установить модуль `fastparquet`, это позволит еще быстрее считывать данные).

In [2]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score

import catboost as cb
import lightgbm as lgb

pd.set_option('display.max_columns', None)

# если у вас есть CUDA, то она понадобится там для экспериментов в catboost
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

sys.path.append('../')

### 1. Reading Data

* В данном соревновании участникам предстоит работать с таким форматом данных, как `Parquet`. Узнать подробнее об этом формате Вы можете самостоятельно. Самое главное &ndash; это крайне эффективный бинарный формат сжатия данных по колонокам. Однако, для непосредственной работы с данными и построения моделей нам нужно прочитать их и трансформировать в pandas.DataFrame. При этом сделать это эффективно по памяти.

In [3]:
TRAIN_DATA_PATH = "/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data"
TEST_DATA_PATH = "/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/test_data"

TRAIN_TARGET_PATH = "/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_target.csv"

In [5]:
from typing import Dict
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras import Model
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from itertools import combinations

import pickle
import tqdm
from typing import List
import os


def pad_sequence(array: np.ndarray, max_len: int) -> np.ndarray:
    """
    Принимает на вход массив массивов ``array`` и производит padding каждого вложенного массива до ``max_len``.

    Параметры:
    -----------
    array: numpy.ndarray
        Входной массив массивов.
    max_len: int
        Длина, до которой нужно сделать padding вложенных массивов.

    Возвращаемое значение:
    ----------------------
    output: numpy.ndarray
        Выходной массив.
    """
    if isinstance(max_len, float):
        print(max_len)
    output = np.zeros((len(features), max_len))
    output[:, :array.shape[1]] = array
    return output


def truncate(x, num_last_credits: int = 0):
    return pd.Series({"sequences": x.values.transpose()[:, -num_last_credits:]})


def transform_credits_to_sequences(credits_frame: pd.DataFrame,
                                   num_last_credits: int = 0) -> pd.DataFrame:
    """
    Принимает pandas.DataFrame с записями кредитных историй клиентов, сортирует кредиты по клиентам
    (внутри клиента сортирует кредиты от старых к новым), берет ``num_last_credits`` кредитов,
    возвращает новый pandas.DataFrame с двумя колонками: id и sequences.
    Каждое значение в столбце sequences - это массив массивов.
    Каждый вложенный массив - значение одного признака во всех кредитах клиента.
    Всего признаков len(features), поэтому будет len(features) массивов.

    Параметры:
    -----------
    credits_frame: pandas.DataFrame
        Датафрейм с записями кредитных историй клиентов.
    num_last_credits: int, default=0
         Количество кредитов клиента, которые будут включены в выходные данные. Если 0, то берутся все кредиты.

    Возвращаемое значение:
    ----------------------
    output: pandas.DataFrame
        Выходной датафрейм с двумя столбцами: "id", "sequences".
    """
    return credits_frame \
        .sort_values(["id", "rn"]) \
        .groupby(["id"])[features] \
        .apply(lambda x: truncate(x, num_last_credits=num_last_credits)) \
        .reset_index()


def create_padded_buckets(frame_of_sequences: pd.DataFrame, bucket_info: Dict[int, int],
                          save_to_file_path: str = None, has_target: bool = True):
    """
    Реализует Sequence Bucketing технику для обучения рекуррентных нейронных сетей.
    Принимает на вход датафрейм ``frame_of_sequences`` с двумя столбцами: "id", "sequences"
    (результат работы функции transform_credits_to_sequences),
    словарь ``bucket_info``, где для последовательности каждой длины указано, до какой максимальной длины нужно делать
    padding, группирует кредиты по бакетам (на основе длины), производит padding нулями и сохраняет результат
    в pickle файл, если требуется.

    Параметры:
    -----------
    frame_of_sequences: pandas.DataFrame
        Входной датафрейм с двумя столбцами "id", "sequences" (результат работы функции transform_credits_to_sequences).
    bucket_info: Dict[int, int]
        Cловарь, где для последовательности каждой длины указано, до какой максимальной длины нужно делать padding.
    save_to_file_path: str, default=None
        Опциональный путь до файла, куда нужно сохранить результат. Если None, то сохранение не требуется.
    has_target: bool, deafult=True
        Флаг, есть ли в frame_of_sequences целевая переменная или нет. Если есть, то она также будет записана в выходной словарь.

    Возвращаемое значение:
    ----------------------
    dict_result: dict
        Выходной словарь со ключами:  "id", "padded_sequences", "target".
    """
    frame_of_sequences["sequence_length"] = frame_of_sequences["sequences"].apply(lambda x: len(x[1]))
    frame_of_sequences["bucket_idx"] = frame_of_sequences["sequence_length"].map(bucket_info)
    padded_seq = []
    targets = []
    ids = []

    for size, bucket in tqdm(frame_of_sequences.groupby("bucket_idx"), desc="Extracting buckets"):
        padded_sequences = bucket["sequences"].apply(lambda x: pad_sequence(x, size)).values
        padded_seq.append(np.stack(padded_sequences, axis=0))

        if has_target:
            targets.append(bucket["flag"].values)

        ids.append(bucket["id"].values)

    frame_of_sequences.drop(columns=["bucket_idx"], inplace=True)

    dict_result = {
        "id": np.array(ids, dtype=np.object),
        "padded_sequences": np.array(padded_seq, dtype=np.object),
        "target": np.array(targets, dtype=np.object) if targets else []
    }

    if save_to_file_path:
        with open(save_to_file_path, "wb") as f:
            pickle.dump(dict_result, f)
    return dict_result


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0, num_parts_to_read: int = 2,
                                    columns: List[str] = None, verbose: bool = False) -> pd.DataFrame:
    """
    Читает ``num_parts_to_read`` партиций и преобразует их к pandas.DataFrame.

    Параметры:
    -----------
    path_to_dataset: str
        Путь до директории с партициями.
    start_from: int, default=0
        Номер партиции, с которой начать чтение.
    num_parts_to_read: int, default=2
        Число партиций, которые требуется прочитать.
    columns: List[str], default=None
        Список колонок, которые нужно прочитать из каждой партиции. Если None, то считываются все колонки.

    Возвращаемое значение:
    ----------------------
    frame: pandas.DataFrame
        Прочитанные партиции, преобразованные к pandas.DataFrame.
    """

    res = []
    start_from = max(0, start_from)
    # dictionory of format {partition number: partition filename}
    dataset_paths = {int(os.path.splitext(filename)[0].split("_")[-1]): os.path.join(path_to_dataset, filename)
                     for filename in os.listdir(path_to_dataset)}
    chunks = [dataset_paths[num] for num in sorted(dataset_paths.keys()) if num >= start_from][:num_parts_to_read]

    if verbose:
        print("Reading chunks:", *chunks, sep="\n")
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        chunk = pd.read_parquet(chunk_path, columns=columns)
        res.append(chunk)
    return pd.concat(res).reset_index(drop=True)


def create_buckets_from_credits(path_to_dataset, bucket_info, save_to_path, frame_with_ids=None,
                                num_parts_to_preprocess_at_once: int = 1,
                                num_parts_total=50, has_target=False):
    block = 0
    for step in tqdm.notebook.tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                   desc="Preparing credit data"):
        credits_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once,
                                                        verbose=True)
        credits_frame.loc[:, features] += 1
        seq = transform_credits_to_sequences(credits_frame)
        print("Transforming credits to sequences is done.")

        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on="id")

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = "00" + block_as_str
        else:
            block_as_str = "0" + block_as_str

        processed_fragment = create_padded_buckets(seq, bucket_info=bucket_info, has_target=has_target,
                                                   save_to_file_path=os.path.join(save_to_path,
                                                                                  f"processed_chunk_{block_as_str}.pkl"))
        block += 1





* Для примера прочитаем одну партицию в память и оценим, сколько RAM занимает весь DataFrame

In [9]:
data_frame = read_parquet_dataset_from_local(TRAIN_DATA_PATH, start_from=0, num_parts_to_read=1)

memory_usage_of_frame = data_frame.memory_usage(index=True).sum() / 10**9
expected_memory_usage = memory_usage_of_frame * 12
print(f"Объем памяти в RAM одной партиции данных с кредитными историями: {round(memory_usage_of_frame, 3)} GB")
print(f"Ожидаемый размер в RAM всего датасета: {round(expected_memory_usage, 3)} GB")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:163: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Объем памяти в RAM одной партиции данных с кредитными историями: 0.964 GB
Ожидаемый размер в RAM всего датасета: 11.564 GB


* Итого, при чтении всех данных сразу, они займут значительный объем памяти. Решение &ndash; читать данные итеративно небольшими чанками. Чанки организованы таким образом, что для конкретного клиента вся информация о его кредитной истории до момента подачи заявки на кредит расположена внутри одного чанка. Это позволяет загружать данные в память небольшими порциями, выделять все необходимые признаки и получать результирующий фрейм для моделирования. Для этих целей мы предоставляем вам функцию `utils.read_parquet_dataset_from_local`.

Так же все чанки данных упорядочены по возрастанию даты заявки на кредит.

In [10]:
del data_frame
gc.collect()

64

In [11]:
train_target = pd.read_csv(TRAIN_TARGET_PATH)
train_target

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


In [12]:
cv = KFold(n_splits=7, random_state=100, shuffle=True)

In [13]:
for fold_, (train_idx, val_idx) in enumerate(cv.split(train_target)):
    train, val = train_target.iloc[train_idx], train_target.iloc[val_idx]
    train.to_csv(f"train_target_fold_{fold_ + 1}.csv", index=False)
    val.to_csv(f"val_target_fold_{fold_ + 1}.csv", index=False)